In [1]:
!pip install pandas

You should consider upgrading via the 'C:\Users\rockd\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [12]:
heart = pd.read_csv('../data/heart_2020_cleaned.csv')

In [53]:
heart

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,False,16.60,True,False,False,3.0,30.0,False,Female,55-59,White,True,True,3,5.0,True,False,True
1,False,20.34,False,False,True,0.0,0.0,False,Female,80 or older,White,False,True,3,7.0,False,False,False
2,False,26.58,True,False,False,20.0,30.0,False,Male,65-69,White,True,True,1,8.0,True,False,False
3,False,24.21,False,False,False,0.0,0.0,False,Female,75-79,White,False,False,2,6.0,False,False,True
4,False,23.71,False,False,False,28.0,0.0,True,Female,40-44,White,False,True,3,8.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,True,27.41,True,False,False,7.0,0.0,True,Male,60-64,Hispanic,True,False,1,6.0,True,False,False
319791,False,29.84,True,False,False,0.0,0.0,False,Male,35-39,Hispanic,False,True,3,5.0,True,False,False
319792,False,24.24,False,False,False,0.0,0.0,False,Female,45-49,Hispanic,False,True,2,6.0,False,False,False
319793,False,32.81,False,False,False,0.0,0.0,False,Female,25-29,Hispanic,False,False,2,12.0,False,False,False


In [52]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  bool   
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  bool   
 3   AlcoholDrinking   319795 non-null  bool   
 4   Stroke            319795 non-null  bool   
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  bool   
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  bool   
 12  PhysicalActivity  319795 non-null  bool   
 13  GenHealth         319795 non-null  int32  
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  bool   
 16  KidneyDisease     31

In [30]:
# convert all yes/no columns to bool
heart[['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']] = \
    heart[['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']].replace(['Yes', 'No'], value=[True, False])

heart[['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']] = \
    heart[['HeartDisease', 'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Diabetic', 'PhysicalActivity', 'Asthma', 'KidneyDisease', 'SkinCancer']].astype(bool)

In [44]:
# associate int values to general health strings
heart['GenHealth'].replace(['Excellent', 'Very good', 'Good', 'Fair', 'Poor'], value=[4, 3, 2, 1, 0], inplace=True)
heart['GenHealth'] = heart['GenHealth'].astype(int)

In [49]:
heart.to_csv('../data/heart_cleaned.csv')